# Marketing Mix Modeling

In this notebook, we will look at Marketing Mix Models as an example of the usage of linear regression in Business Intelligence.

The task: *We want to model sales (or another KPI such as customers, leads, etc.) as a function of advertising, trends, events, seasonality, price, distribution, macro-economic factors, the weather etc.*

Notes:
* For offline medias, it can be really hard to measure the effect of marketing on sales
* The different variables comes with different controls from our side
    + Seasonality, macro-economic factors and the weather, we cannot control
    + Price and distribution can be controlled to some extend
    + Advertising is where we have the most control
* With such a model, we can evaluate marketing campaigns and plan future campaign based on which marketing mix that we predicted to generate most sales
* A classical approach is to use some sort of Linear Regression.
* See the blogs: https://analyticsartist.wordpress.com/2014/08/17/marketing-mix-modeling-explained-with-r/ and https://analyticsartist.wordpress.com/2013/11/02/calculating-adstock-effect/

In [1]:
library(tidyverse)
options(repr.plot.width=8, repr.plot.height=5)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.4
✔ tibble  1.4.1     ✔ dplyr   0.7.4
✔ tidyr   0.7.2     ✔ stringr 1.3.1
✔ readr   1.1.1     ✔ forcats 0.2.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


## A simple first model

The simplest model we can think off: `Sales = Base + b_1*Advertising`

In [2]:
sales <- c(37, 89, 82, 58, 110, 77, 103, 78, 95, 106, 98, 96, 68, 96, 157, 198, 145, 132, 96, 135)
ad <- c(6, 27, 0, 0, 20, 0, 20, 0, 0, 18, 9, 0, 0, 0, 13, 25, 0, 15, 0, 0)
modFit0 <- lm(sales~ad)
summary(modFit0)
ggplot(mapping = aes(x = ad, y = sales)) + geom_point() + geom_smooth(method = "lm")


Call:
lm(formula = sales ~ ad)

Residuals:
    Min      1Q  Median      3Q     Max 
-63.153 -15.048  -7.746   8.457  67.364 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  90.5267     9.7277   9.306 2.67e-08 ***
ad            1.6044     0.7957   2.016    0.059 .  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 33.93 on 18 degrees of freedom
Multiple R-squared:  0.1842,	Adjusted R-squared:  0.1389 
F-statistic: 4.065 on 1 and 18 DF,  p-value: 0.05895


## Adding adstock transformation

The effect of an ad is not limited to the week (or day) the add is shown
- The effect *carries over* to the following weeks (days) with a decay though
- Explanation of this effect: Exposures to an add build awareness in consumers' minds. This awareness doesn't disappear right away, but remains in the consumers' memory. However, memory decays over time.
- The adstock formula is: `AdEffect(t) = AdSpend(t) + CarryOver * AdEffect(t-1)`

In [3]:
ad_adstock <- as.numeric(stats::filter(x=ad, filter=.50, method="recursive"))
ggplot(mapping = aes(x = seq(1,length(ad)), y = ad)) +
  geom_col() + ylab("Advertising") + xlab("Time (in Weeks)") +
  geom_line(mapping = aes(x = seq(1,length(ad)), y = ad_adstock), size = 2, color = "blue")

For different values of the `CarryOver` parameter we get fairly different awareness effect:

In [4]:
ad_adstock <- as.numeric(stats::filter(x=ad, filter=.25, method="recursive"))
ggplot(mapping = aes(x = seq(1,length(ad)), y = ad)) +
  geom_col() + ylab("Advertising") + xlab("Time (in Weeks)") +
  geom_line(mapping = aes(x = seq(1,length(ad)), y = ad_adstock), size = 2, color = "blue") +
  ggtitle("Carry-over of 0.25")

ad_adstock <- as.numeric(stats::filter(x=ad, filter=.75, method="recursive"))
ggplot(mapping = aes(x = seq(1,length(ad)), y = ad)) +
  geom_col() + ylab("Advertising") + xlab("Time (in Weeks)") +
  geom_line(mapping = aes(x = seq(1,length(ad)), y = ad_adstock), size = 2, color = "blue") +
  ggtitle("Carry-over of 0.75")

## Adding adstock transformation

We can now add the adstock transformation of the advertising to the mode. With the ad stock transformation, the model looks like: `Sales = Base + b1*Adstock(Advertising, a)` Here `a` is the adstock parameter that determines how strong the carry-over should be.

In [5]:
ad_adstock <- as.numeric(stats::filter(x=ad, filter=.50, method="recursive"))
modFit1 <- lm(sales ~ ad_adstock)
summary(modFit1)
ggplot(mapping = aes(x = ad_adstock, y = sales)) + geom_point() + geom_smooth(method = "lm")


Call:
lm(formula = sales ~ ad_adstock)

Residuals:
   Min     1Q Median     3Q    Max 
-46.84 -20.57 -10.54  14.16  59.23 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  71.2232    14.7331   4.834 0.000133 ***
ad_adstock    2.1034     0.8538   2.464 0.024057 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 32.49 on 18 degrees of freedom
Multiple R-squared:  0.2522,	Adjusted R-squared:  0.2106 
F-statistic: 6.069 on 1 and 18 DF,  p-value: 0.02406


## Adding another media

We can have multiple medias in the model. * For instance, the formula can look like: `Sales = Base + b_1*Advertising_1 + b_2*Advertising_2`. Note, the different medias/advertisements could have different carry overs.

In [6]:
ad1 <- ad
ad2 <- c(3, 0, 4, 0, 5, 0, 0, 0, 8, 0, 0, 5, 0, 11, 16, 11, 5, 0, 0, 15)
ad1_adstock <- as.numeric(stats::filter(x=ad1, filter=.3, method="recursive"))
ad2_adstock <- as.numeric(stats::filter(x=ad2, filter=.3, method="recursive"))
modFit2 <- lm(sales~ad1_adstock+ad2_adstock)
summary(modFit2)


Call:
lm(formula = sales ~ ad1_adstock + ad2_adstock)

Residuals:
    Min      1Q  Median      3Q     Max 
-44.791 -10.432   2.156   8.657  28.317 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  56.8937     7.6794   7.409 1.02e-06 ***
ad1_adstock   1.8808     0.4681   4.018 0.000892 ***
ad2_adstock   4.5375     0.7163   6.335 7.47e-06 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 18.56 on 17 degrees of freedom
Multiple R-squared:  0.7694,	Adjusted R-squared:  0.7423 
F-statistic: 28.36 on 2 and 17 DF,  p-value: 3.837e-06


## Adding trend

There might be a trend in the response variable sales (it can be due to several things). Let us a have look at sales over time:

In [7]:
ggplot(mapping = aes(x = 1:length(sales), y = sales)) + geom_line()

It is a bit hard to see from this plot whether there is a trend. It could look a bit like an upward trend though. 

Let us have a look at the residual plots for the two models instead:

In [8]:
ggplot(mapping = aes(x = 1:length(modFit1$residuals), y = modFit1$residuals)) + geom_point()
ggplot(mapping = aes(x = 1:length(modFit2$residuals), y = modFit2$residuals)) + geom_point()

Here the trend seems much clearer and it would probably make sense to add a trend to the model. So let us add trend to the model:

In [9]:
trend <- 1:length(sales)
modFit3 <- lm(sales~trend+ad1_adstock+ad2_adstock)
summary(modFit3)


Call:
lm(formula = sales ~ trend + ad1_adstock + ad2_adstock)

Residuals:
    Min      1Q  Median      3Q     Max 
-18.661  -5.723   1.234   6.537  14.065 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  30.6649     5.5836   5.492 4.92e-05 ***
trend         3.0271     0.4467   6.776 4.45e-06 ***
ad1_adstock   2.1706     0.2490   8.718 1.79e-07 ***
ad2_adstock   2.9819     0.4400   6.777 4.44e-06 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 9.727 on 16 degrees of freedom
Multiple R-squared:  0.9404,	Adjusted R-squared:  0.9292 
F-statistic: 84.17 on 3 and 16 DF,  p-value: 5.165e-10


Let us look at the residuals again:

In [11]:
ggplot(mapping = aes(x = 1:length(modFit3$residuals), y = modFit3$residuals)) + geom_point()
ggplot() + geom_histogram(aes(x = modFit3$residuals), binwidth = 4)

## Interpreting the results

Let us now start interpreting the results.

First we will simply sum the effect of each of the variables ("base", "ad1_adstock", "ad2_adstock", "trend") to get an idea about how much the sales was driven by each one of the in the period in total.

In [ ]:
total_effect <- data.frame(Variable = c("base", "ad1_adstock", "ad2_adstock", "trend"),
                           Effect = c(modFit3$coefficients["(Intercept)"] * length(sales),
                                      sum(modFit3$coefficients["ad1_adstock"] * ad1_adstock),
                                      sum(modFit3$coefficients["ad2_adstock"] * ad2_adstock),
                                      sum(modFit3$coefficients["trend"] * trend)))
ggplot(total_effect, aes(Variable, Effect)) + geom_col() + ggtitle("Total effect")

From this plot we can see that most of the sales is driven by a base sales and a trend. However, the two ad campaigns did also drive a significant amount of sales. Ad1 seems to have driven more sales than ad2.

We can also sum the two original variables `ad1` and `ad2` as they represented that actual spend on the two medias. Thus we will for each ad get the total spend in the period.

In [ ]:
total_spend <- data.frame(Media = c("ad1", "ad2"),
                          Spend = c(sum(ad1), sum(ad2)))
ggplot(total_spend, aes(Media, Spend)) + geom_col() + ggtitle("Total Media Spend")

So while, ad1 drove more sales than ad2, we also spend more money on ad1 than on ad2. Thus we should find another more precise way of comparing the two ads.

One such way of comparing could be on "Cost per sales" (CPS), that is for each of the two ad, what does it cost us to generate one extra sale?

In [ ]:
cps_df <- data.frame(Media = c("ad1", "ad2"),
                     CPS = c(total_spend$Spend[1]/sum(total_effect$Effect[2]),
                             total_spend$Spend[2]/sum(total_effect$Effect[3])))

ggplot(cps_df, aes(Media, CPS)) + geom_col() + ggtitle("Cost per Sales (CPS)")

Here we see that it cost us more to generate an extra sale with ad1 than with ad2. In this sense ad2 seems more effective.

Another way of evaluating the two medias would be on "return on investment" (ROI). So let us do this.

The formula for ROI is:

`ROI = (gain from investment - cost of investment) / cost of investment`

To use this, we need to know what the "gain from investment" is. This is the profit margin (What the company makes on each sold product). For now, we will assume the profit margin is `0.5` EUROs. Then we can calculate the total ROIs of each ad:

In [ ]:
roi_df <- data.frame(Media = c("ad1", "ad2"),
                          ROI = c((0.5 * sum(modFit3$coefficients["ad1_adstock"] * ad1_adstock) -
                                     total_spend$Spend[1])/total_spend$Spend[1],
                                  (0.5 * sum(modFit3$coefficients["ad2_adstock"] * ad1_adstock) -
                                     total_spend$Spend[2])/total_spend$Spend[2]))
ggplot(roi_df, aes(Media, ROI)) + geom_col() + ggtitle("Return on Investment (ROI)")

From this we can see that for every extra dollar we invest in ad1, we will make profit of around 0.6 cents, while for every dollar extra we invest in ad2, we will make a profit of almost 3 dollars. Thus, ad2 is much more efficient in this sense.

A note is in order here. It seems like we could just invest all the money we can get in ad2 and thereby increase our profit threefold. However, there is also a saturation effect to advertisement we have not included in the model. At some point the market is saturated and we cannot sell more products or there are simply no more room in the media for our ad (all tv spots is full of it). Thus, under large increase in advertisement spend, we are likely to see a drop in ROIs.